In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt, matplotlib.image as mpimg

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

C:\Users\hdc\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hdc\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hdc\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int1

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


mnist.train.images的每个元素就是一张图片，28\*28=784的一维列向量，mnist.train.labels的每个元素就是一个标签，当one_hot是True的时候就是10的一维列向量，是False的时候就是一个标量，代表图片是几

In [2]:
mnist.train.labels[1]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [3]:
mnist.train.images[1]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [4]:
x = tf.placeholder('float32', [None, 28*28], name='x')
y = tf.placeholder('float32', [None, 10], name='y')

创建学习变量、权重和偏置

In [5]:
W = tf.Variable(tf.zeros([784, 10]), name='weight')
bias = tf.Variable(tf.zeros([10]), name='bias')

创建逻辑回归模型

In [6]:
y_hat = tf.nn.softmax(tf.matmul(x, W)+bias)

定义损失函数

In [7]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=y_hat))

添加summary

In [8]:
w_h = tf.summary.histogram('weights', W)
b_h = tf.summary.histogram('bias', bias)

tf.summary.scalar('loss', loss)

<tf.Tensor 'loss:0' shape=() dtype=string>

定义优化器

In [9]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

初始化变量

In [10]:
init = tf.global_variables_initializer()

组合所有的summary操作

In [11]:
merged_summary_op = tf.summary.merge_all()

定义评估函数

In [12]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), tf.argmax(y_hat,1)), 'float32'))

定义会话并将所有的summary存储在定义的文件夹中

In [13]:
with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.summary.FileWriter('log/mnist_soft_regression', sess.graph)
    
    # train
    for epoch in range(200):
        loss_avg = 0
        batch_size = 256
        num_of_batch = int(mnist.train.num_examples/batch_size)
        for i in range(num_of_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, l, summary_str = sess.run([optimizer, loss, merged_summary_op], feed_dict={x: batch_x, y: batch_y})
            loss_avg += l
            summary_writer.add_summary(summary_str, epoch*batch_size+i)
        loss_avg = loss_avg / num_of_batch
        print('Epoch {0}: Loss {1}'.format(epoch, loss_avg))
    
    print('Finished')
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))

Epoch 0: Loss 2.2884286775767246
Epoch 1: Loss 2.2429473946027665
Epoch 2: Loss 2.183812643880042
Epoch 3: Loss 2.11294673647836
Epoch 4: Loss 2.046925547524033
Epoch 5: Loss 1.9970453114153068
Epoch 6: Loss 1.9565557029759773
Epoch 7: Loss 1.9223848380775095
Epoch 8: Loss 1.8941964763347234
Epoch 9: Loss 1.8707730513866816
Epoch 10: Loss 1.8503968058345475
Epoch 11: Loss 1.8340228077407195
Epoch 12: Loss 1.8200792224607736
Epoch 13: Loss 1.807920060982214
Epoch 14: Loss 1.7985001763450765
Epoch 15: Loss 1.78774631636165
Epoch 16: Loss 1.7815833587512793
Epoch 17: Loss 1.773886711240929
Epoch 18: Loss 1.76756207408192
Epoch 19: Loss 1.7617986441772675
Epoch 20: Loss 1.7573749272622794
Epoch 21: Loss 1.7528326511383057
Epoch 22: Loss 1.7477481198088032
Epoch 23: Loss 1.744238089735263
Epoch 24: Loss 1.7408750241047868
Epoch 25: Loss 1.7374826730968795
Epoch 26: Loss 1.7328503065020124
Epoch 27: Loss 1.7321737765151763
Epoch 28: Loss 1.7281159836555195
Epoch 29: Loss 1.7264692387848257
E